<a href="https://colab.research.google.com/github/nicolasquant/inteligencia_artificial/blob/main/ML_knn_git_Amanda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

#importando a nossa base de dados
df = pd.read_csv('vinho.csv') # note que temos que ter a planilha dos vinhos baixada
df.head() # como no linux, .head printa apenas as 5 primeiras linhas da nossa tabela


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,is good
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0.0


# Nova seção

In [ ]:
# é necessário neste algoritmo deixar os pontos na mesma escala para calcular a distância entre dois pontos (diferença entre as normas)
# deixaremos os dados no intervalo [0,1] utilizando a técnica Min-max Scaling (subtrair o menor valor da atributo e dividir pela amplitude)
def MinMaxScale(x, min, max): # apenas uma função que normaliza os pontos
    return (x - min) / (max - min) # x pode ser um vetor, como no exemplo abaixo

%time # %time é um comendo do IPython onde The CPU and wall clock times are printed, and the value of the expression (if any) is returned.
# wall time é o tempo decorrido entre o início e final de um processo computacional (o tempo será contado a partir daqui e terminará no final do código abaixo)
mins = df.min()
maxs = df.max()

# aplicamos a normalização para todas as colunas da base de dados
for column in df: # acredito que a definição de coluna venha do pandas ou do arquivo csv - aqui o código vai rodar cada coluna nomeada
    df[column] = MinMaxScale(df[column], mins[column], maxs[column])


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [ ]:
#selecionamos qual é a nossa coluna que contem as respostas
target = "is good" # repare que esse coluna é e coluna de "output" - aqui assumimos apenas valores binários
# a coluna 'is good' , então, é nossa saída da rede neural (ou seja, a função real, onde treinaremos nossa função hipotese a partir dela)
features = df.columns.to_list() # aqui estamos criando uma string com o nome de todas colunas
features.remove(target) # aqui estamos removendo a coluna target pois ela não será um parâmetro para nossa rede neural
print(features)


['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']


In [ ]:
# e então separamos os dados em treino e validação

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

df[features]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846
1,0.283186,0.520548,0.00,0.116438,0.143573,0.338028,0.215548,0.494126,0.362205,0.209581,0.215385
2,0.283186,0.438356,0.04,0.095890,0.133556,0.197183,0.169611,0.508811,0.409449,0.191617,0.215385
3,0.584071,0.109589,0.56,0.068493,0.105175,0.225352,0.190813,0.582232,0.330709,0.149701,0.215385
4,0.247788,0.397260,0.00,0.068493,0.106845,0.140845,0.098940,0.567548,0.606299,0.137725,0.153846
...,...,...,...,...,...,...,...,...,...,...,...
1594,0.141593,0.328767,0.08,0.075342,0.130217,0.436620,0.134276,0.354626,0.559055,0.149701,0.323077
1595,0.115044,0.294521,0.10,0.089041,0.083472,0.535211,0.159011,0.370778,0.614173,0.257485,0.430769
1596,0.150442,0.267123,0.13,0.095890,0.106845,0.394366,0.120141,0.416300,0.535433,0.251497,0.400000
1597,0.115044,0.359589,0.12,0.075342,0.105175,0.436620,0.134276,0.396476,0.653543,0.227545,0.276923


In [ ]:
# a função para o calculo das distâncias, conforme comentado anteriormente, será a distância euclidiana (l²)
def distance(x, y):
    return np.sqrt(sum((x - y)**2))

In [ ]:
# defnimos então nossa classe KNN para conter nossa IA

class KNearestNeighbors: # nessa classe guardaremos apenas os vixinhos mais próximos
    # inicia a variável que guarda a quantidade de vizinhos proximos
    def __init__(self, k):
        self.k = k # k é o nosso hyperparâmetro: ou seja, a quantidade de médias que calcularemos para afirmar se nosso 'ponto novo pertence a um grupo ou outro

    # disposição dos pontos no espaço
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_val):
        # Lista para resultado
        y_pred = [] # y_pred é o y de predição. Ou seja, o y que iremos prever após o calculo das médias
        # Para cada amostra de vale, vamos calcular sua distância até os dados de treino
        X_val_arr = X_val.to_numpy() # X_val é a matriz com os elementos de X de validação - o comando .to_numpy converte os dados do tipo dataframe como uma array no numpy
        X_train_arr = self.X_train.to_numpy()
        for row_val in X_val_arr:
            # Array para guardar [index_train, distancia]
            distances = []
            # Vamos percorrer os dados de treino colocando as distancias em nossa matriz
            for index_train, row_train in enumerate(X_train_arr):
                distances.append([index_train, distance(row_val, row_train)])
            # Vamos ordenar o array com base nas distancias (np.argsort retorna somente os indices)
            distances = sorted(distances, key=lambda x: x[1]) #sorted ordena nossos dados em ordem crescente
            # Pegando os k primeiros vizinhos
            nearestNeighbors = distances[0:self.k]
            # Ignorando agora as distancias
            nearestNeighbors = np.array(nearestNeighbors)[:,0]
            # Para cada vizinho, vamos analisar o target nod dados de treino
            result = []
            for neighbor in nearestNeighbors:
                result.append(y_train.to_numpy()[int(neighbor)])
            if np.array(result).sum() > len(result) / 2: # aqui, estamos dizendo que se a soma dos elementos dentro da array de resultados for maior que  o tamanho da lista de resultados, então nossa classe é a 1, caso contrário, a 0
                y_pred.append(1)
            else:
                y_pred.append(0)

        return y_pred # y_pred nos dá os resultados das classes na quais chegamos a partir do calculo das distancias

    def accuracy(self, y_val, y_pred):
        total = 0
        for val, pred in zip(y_val, y_pred):
            if (val == pred):
                total += 1
        return total / len(y_val), print(len(y_val))

In [ ]:
# definimos então um modelo
model = KNearestNeighbors(k =5)
# realizamos o treinamento
model.fit(X_train, y_train) # repare que após definirmos uma função dentro de uma classe, podemos chamar a função a partir da classe, como fazemos com as bibliotecas
#predição
y_pred = model.predict(X_val)
# e finalmente teste da precisão
model.accuracy(y_pred, y_val)

320


(0.684375, None)

In [ ]:
#podemos também ver o comportamento do modelo para difentes valores de K
ks = [3, 5, 7, 9]
for k in ks:
  model = KNearestNeighbors(k=k)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)
  print(model.accuracy(y_pred, y_val))

320
(0.71875, None)
320
(0.684375, None)
320
(0.709375, None)
320
(0.7, None)
